## Detalles iniciales

In [276]:
# NO EDITAR ESTA CELDA
SMALL_CHAT_MODEL_CON_CASTELLANO =  "Qwen/Qwen1.5-1.8B-Chat"
DEVICE = "cpu"

from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained(SMALL_CHAT_MODEL_CON_CASTELLANO).to(DEVICE)
tokenizer = AutoTokenizer.from_pretrained(SMALL_CHAT_MODEL_CON_CASTELLANO)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## Pre-procesado

In [277]:
# En esta celda crearemos la funcion para pre-procesar el texto si es necesario
def preprocesar_texto(texto: str) -> str:
    texto = texto.strip()  # Remove leading and trailing whitespace
    return texto

texto_ejemplo = "  La película Titanic, dirigida por James Cameron y estrenada en 1997, cuenta la trágica historia de un romance en el famoso barco.  "
texto_preprocesado = preprocesar_texto(texto_ejemplo)
print(texto_preprocesado)

La película Titanic, dirigida por James Cameron y estrenada en 1997, cuenta la trágica historia de un romance en el famoso barco.


## Prompt

In [ ]:
# Funcion crear_prompt
def crear_prompt(
        mensaje_usuario: str,
        tokenizer: AutoTokenizer,
        historial: str = None,
        mensaje_sistema: str = "You are the best text analysis assistant."
    ) -> str:
    if historial is None:
        messages = [{"role": "system", "content": mensaje_sistema}]
    else:
        messages = []
        
    messages.append({"role": "user", "content": mensaje_usuario})
    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    if historial is not None:
        prompt = f"{historial} {prompt}"
    return prompt


In [279]:
msj = f"  La película Titanic, dirigida por James Cameron y estrenada en 1997, cuenta la trágica historia de un romance en el famoso barco.  " 
prompt = crear_prompt(msj, tokenizer)
print(prompt)

<|im_start|>system
You are the best text analysis assistant.<|im_end|>
<|im_start|>user
  La película Titanic, dirigida por James Cameron y estrenada en 1997, cuenta la trágica historia de un romance en el famoso barco.  <|im_end|>
<|im_start|>assistant



In [280]:
# Example usage
descripcion_peli = "'Inception' (Origen), dirigida por Christopher Nolan en 2010, es un thriller de ciencia ficción que explora el robo de secretos a través del subconsciente durante el sueño, llevando a cabo una idea compleja: la inserción de otra idea."

prompt = crear_prompt(descripcion_peli, tokenizer)
print(prompt)

<|im_start|>system
You are the best text analysis assistant.<|im_end|>
<|im_start|>user
'Inception' (Origen), dirigida por Christopher Nolan en 2010, es un thriller de ciencia ficción que explora el robo de secretos a través del subconsciente durante el sueño, llevando a cabo una idea compleja: la inserción de otra idea.<|im_end|>
<|im_start|>assistant



## Respuesta

In [281]:
from transformers import TextStreamer
from transformers.generation.utils import GenerateDecoderOnlyOutput, GenerateBeamDecoderOnlyOutput
import torch

def responde_al_mensaje(
        model: AutoModelForCausalLM,
        tokenizer: AutoTokenizer,
        prompt: str,
        streamer: TextStreamer = None,
        gen_kwargs: dict = None,
        return_scores: bool = False,
    ) -> list[str] | tuple[list[str], GenerateDecoderOnlyOutput]:

    gen_kwargs = gen_kwargs or {}
    if return_scores:
        gen_kwargs["return_dict_in_generate"] = True
        gen_kwargs["output_scores"] = True

    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(DEVICE)

    model_output = model.generate(
        input_ids,
        **gen_kwargs,
        streamer=streamer,
    )
    if isinstance(model_output, GenerateDecoderOnlyOutput):
        # Será el caso si en model.generate hemos pasado return_dict_in_generate=True
        response_tokens = model_output.sequences
    elif isinstance(model_output, torch.Tensor):
        response_tokens = model_output
    elif isinstance(model_output, GenerateBeamDecoderOnlyOutput):
        response_tokens = model_output.sequences
    else:
        raise ValueError(f"El modelo ha devuelto un tipo inesperado: {type(model_output)}")


    responses_txt = tokenizer.batch_decode(response_tokens[:,len(input_ids[0]):], skip_special_tokens=True)

    return responses_txt, model_output


## Estrategias Prompting

In [282]:
# Ejemplos de "test" con los que trabajaremos
peli1 = """El Laberinto del Fauno', dirigida por Guillermo del Toro y estrenada en 2006, mezcla la fantasía oscura con la historia real de la posguerra española, siguiendo a una niña que descubre un mundo mágico y peligroso."""
peli2= """Inception' (Origen), dirigida por Christopher Nolan en 2010, es un thriller de ciencia ficción que explora el robo de secretos a través del subconsciente durante el sueño, llevando a cabo una idea compleja: la inserción de otra idea."""

In [283]:
# Enfoque más directo

from datetime import datetime, timedelta
import locale
# locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8');


def crear_prompt_chat_model_films(peli: str, tokenizer: AutoTokenizer):
    msj = f"""
Eres un asistente que proporciona información sobre películas. Recibes descripciones de películas y tu función es SOLAMENTE extraer la información proporcionada en los textos. No generas información adicional. La información la darás en un formato estructurado del tipo:
    {{
        "Título": "<título de la película>",
        "Director": "<nombre del director o directora>",
        "Año estreno": "<año de estreno>"
    }}
Indicación adicional:
    - Si no se especifica algún campo, se debe incluir un string vacío ""
    - Antes de dar una respuesta, tienes que pensar paso a paso
    
Has recibido esta descripcion:
{peli}
"""
    prompt = crear_prompt(msj, tokenizer)
    return prompt

prompt = crear_prompt_chat_model_films(peli2, tokenizer)
print(prompt)

<|im_start|>system
You are the best text analysis assistant.<|im_end|>
<|im_start|>user

Eres un asistente que proporciona información sobre películas. Recibes descripciones de películas y tu función es SOLAMENTE extraer la información proporcionada en los textos. No generas información adicional. La información la darás en un formato estructurado del tipo:
    {
        "Título": "<título de la película>",
        "Director": "<nombre del director o directora>",
        "Año estreno": "<año de estreno>"
    }
Indicación adicional:
    - Si no se especifica algún campo, se debe incluir un string vacío ""
    - Antes de dar una respuesta, tienes que pensar paso a paso
    
Has recibido esta descripcion:
Inception' (Origen), dirigida por Christopher Nolan en 2010, es un thriller de ciencia ficción que explora el robo de secretos a través del subconsciente durante el sueño, llevando a cabo una idea compleja: la inserción de otra idea.
<|im_end|>
<|im_start|>assistant



In [284]:
"""# Veamos el resultado
SEED = 123
gen_kwargs = {"max_new_tokens": 50, "temperature": 0.2}
response, model_output = responde_al_mensaje(model, tokenizer, crear_prompt_chat_model_films(peli1, tokenizer), streamer=streamer, gen_kwargs=gen_kwargs)
print("-"*30)
response, model_output = responde_al_mensaje(model, tokenizer, crear_prompt_chat_model_films(peli2, tokenizer), streamer=streamer, gen_kwargs=gen_kwargs)"""

'# Veamos el resultado\nSEED = 123\ngen_kwargs = {"max_new_tokens": 50, "temperature": 0.2}\nresponse, model_output = responde_al_mensaje(model, tokenizer, crear_prompt_chat_model_films(peli1, tokenizer), streamer=streamer, gen_kwargs=gen_kwargs)\nprint("-"*30)\nresponse, model_output = responde_al_mensaje(model, tokenizer, crear_prompt_chat_model_films(peli2, tokenizer), streamer=streamer, gen_kwargs=gen_kwargs)'

In [286]:
def crear_prompt_chat_model_films_few_shot(mail: str, tokenizer: AutoTokenizer) -> str:
    ejemplos = [
        {
            "texto": "El buscador del Fauno, dirigida por Beatriz Morales y estrenada en 2006, mezcla la fantasía con la historia real de la posguerra española, siguiendo a una niña que descubre un mundo mágico.",
            "respuesta": {
                "Título": "El buscador del Fauno",
                "Director": "Beatriz Morales",
                "Año estreno": "2006"
            }
        },
        {
            "texto": "Castilla y Leones, dirigida por Cristo Pedro Nolan en 2010, es un thriller de ciencia ficción que explora el robo de secretos a través del subconsciente durante el sueño, llevando a cabo una idea compleja: la inserción de otra idea.",
            "respuesta": {
                "Título": "Castilla y Leones",
                "Director": "Cristo Pedro",
                "Año estreno": "2010"
            }
        },
        {
            "texto": "La película Juan es un barco, dirigida por Pedro Luis y estrenada en 1997, cuenta la trágica historia infantil de Juan el barco y sus amigos.",
            "respuesta": {
                "Título": "Juan es un barco",
                "Director": "Pedro Luis",
                "Año estreno": "1997"
            }
        },
        {
            "texto": "Matrix, una película de ciencia ficción dirigida por las hermanas Wachowski y estrenada en 1999, sigue la historia de un programador que descubre la verdadera naturaleza de su realidad.",
            "respuesta": {
                "Título": "Matrix",
                "Director": "Las hermanas Wachowski",
                "Año estreno": "1999"
            }
        },
        {
            "texto": "Dirigida por Martin Roberts en 1990, esta película sigue la vida de un mafioso desde su infancia hasta su ascenso y caída en el mundo del crimen organizado.",
            "respuesta": {
                "Título": "",
                "Director": "Martin Roberts",
                "Año estreno": "1990"
            }
        },
        {
            "texto": "'El padrino', estrenada en 1972, es una épica saga sobre una familia criminal en Nueva York y su lucha por mantener el poder.",
            "respuesta": {
                "Título": "El padrino",
                "Director": "",
                "Año estreno": "1972"
            }
        },
        {
            "texto": "Dirigida por Seven Rodriguez, esta película es un clásico del cine de aventuras, siguiendo a un arqueólogo en su búsqueda de un artefacto sagrado.",
            "respuesta": {
                "Título": "",
                "Director": "Seven Rodriguez",
                "Año estreno": ""
            }
        },
        {
            "texto": "Esta obra maestra del cine, lanzada en 1942, narra una historia de amor y sacrificio en el contexto de la Segunda Guerra Mundial.",
            "respuesta": {
                "Título": "",
                "Director": "",
                "Año estreno": "1942"
            }
        },
        {
            "texto": "Una pelicula de Disney sbre un jovenn león ue debe vencer a tu tío para recuperar su lugar en el trono",
            "respuesta": {
                "Título": "",
                "Director": "",
                "Año estreno": ""
            }
        }
    ]

    ejemplos_str = ""
    for ejemplo in ejemplos:
        ejemplos_str += f"""
Ejemplo:
Descripción de la película: {ejemplo["texto"]}
Respuesta estructurada:
{{
    "Título": "{ejemplo["respuesta"]["Título"]}",
    "Director": "{ejemplo["respuesta"]["Director"]}",
    "Año estreno": "{ejemplo["respuesta"]["Año estreno"]}"
}}
"""

    msj = f"""
You are an assistant that provides information about movie descriptions IN SPANISH. You receive descriptions of movies and your function is to EXPLICITLY extract from the text the following information in a structured format from the variable "movie description" of type:
    {{
        "Título": "<título de la película>",
        "Director": "<nombre del director o directora>",
        "Año estreno": "<año de estreno>"
    }}
Additional information:
    - Do not invent information that does not appear directly in the text.
    - If no field is specified, an empty string "" must be included
    

{ejemplos_str}

Has recibido esta descripción:
{mail}
"""
    prompt = crear_prompt(msj, tokenizer)
    return prompt



## Lista -> diccionario

In [ ]:
# Esta función para convertir un string en diccionario te seŕa útil.
import ast
import json
import re

def list_to_dict(lst: list) -> dict:
    if not lst:
        return {"Título": "", "Director": "", "Año estreno": ""}
    
    s = lst[0].strip()
    
    # Reemplazar todos los {{ en una fila por solo uno {
    s = re.sub(r'\{+', '{', s)
    
    if not s:
        return {"Título": "", "Director": "", "Año estreno": ""}
    
    try:
        # Intentar convertir el string a un diccionario
        s_dict = json.loads(s)
    except json.JSONDecodeError as e:
        print(f"Error al evaluar el string como un diccionario: {e}")
        s_dict = {"Título": "", "Director": "", "Año estreno": ""}
    
    return s_dict

# Ejemplo de uso:
lista = ['{\n    "Título": "Inception",\n    "Director": "Christopher Nolan",\n    "Año estreno": "2010"\n}']
print(list_to_dict(lista))

lista_vacia = ['']
print(list_to_dict(lista_vacia))

{'Título': 'Inception', 'Director': 'Christopher Nolan', 'Año estreno': '2010'}
{'Título': '', 'Director': '', 'Año estreno': ''}


## Post-procesado

In [ ]:
# Postprocesado

def postprocesar_respuesta(
    respuesta: str
) -> dict:
    try:
        respuesta_dict = list_to_dict(respuesta)
    except Exception as e:
        print(f"Error al convertir la respuesta en diccionario: {e}")
        respuesta_dict = {"Título": "", "Director": "", "Año estreno": ""}
    return respuesta_dict

## Pipeline Completo

In [ ]:
# Pipeline_completo()
def pipeline_completo(
    descripcion_peli: str,
    model: AutoModelForCausalLM,
    tokenizer: AutoTokenizer,
    max_new_tokens: int = 50,
    historial: str = None,
    mensaje_sistema: str = "You are the best information summarizer."
) -> dict:
    # Pre-procesar el texto si es necesario
    texto_preprocesado = preprocesar_texto(descripcion_peli)
    
    # Crear prompt
    prompt = crear_prompt_chat_model_films_few_shot(texto_preprocesado, tokenizer)
    
    # Generar respuesta con el modelo
    gen_kwargs = {
        "max_new_tokens": max_new_tokens}
    respuesta_modelo = responde_al_mensaje(
        model=model,
        tokenizer=tokenizer,
        prompt=prompt,
        gen_kwargs=gen_kwargs
    )
    print("Esta es la respuesta del modelo en el pipeline:\n")
    print(respuesta_modelo[0])
    
    
    # Postprocesar respuesta y convertirla en diccionario
    respuesta_dict  = postprocesar_respuesta(respuesta_modelo[0])

    
    return respuesta_dict 


## Evaluación

In [290]:
# NO EDITAR ESTA CELDA  
from typing import Callable
def evaluar_pipeline_con_conjunto_de_test(
    ejemplos: list[dict[str, dict]],
    tu_pipeline: Callable, 
    **pipeline_kwargs) -> float:
    """
     Obtener el ratio de acierto (del 0% al 100%) sobre los ejemplos de test.
    """
    total_campos, total_aciertos = 0, 0
    for i, ejemplo in enumerate(ejemplos):
        texto, resp_correcta = ejemplo["texto"], ejemplo["respuesta"]
        resp = tu_pipeline(texto, **pipeline_kwargs)
        total_campos += 3
        if isinstance(resp, dict): # La salida de tu pipeline debería ser un diccionario
            print(f"Evaluando {i + 1}/{len(ejemplos)}..")
            print(f"Resultado obtenido: {resp}")
            print(f"Resultado esperado: {resp_correcta}")
            if "Título" in resp:
                acierto = int(resp["Título"].strip().lower() == resp_correcta["Título"].strip().lower())
                if acierto:
                    print(f"Título correcto!")
                total_aciertos += acierto
            if "Director" in resp:
                acierto = int(resp["Director"].strip().lower() == resp_correcta["Director"].strip().lower())
                if acierto :
                    print(f"Director correcto!")
                total_aciertos += acierto
            if "Año estreno" in resp:
                acierto = int(str(resp["Año estreno"]) ==
                              str(resp_correcta["Año estreno"]))
                if acierto:
                    print(f"Año de estreno correcto!")
                total_aciertos += acierto
        print(f"Tasa de acierto acumulada:{total_aciertos/total_campos:.1%}")
    tasa_aciertos = total_aciertos / total_campos
    print(f"El ratio de aciertos es del {tasa_aciertos*100:.1f}%")
    
    return tasa_aciertos


In [291]:
TEXTOS_TEST = [
    {
      "texto": "En *El Resplandor*, dirigida por Stanley Kubrick, un escritor se convierte en el cuidador de invierno de un aislado hotel de montaña, donde descubre horrores sobrenaturales que afectan a su familia. La película se estrenó en 1980.",
      "respuesta": {
        "Título": "El Resplandor",
        "Director": "Stanley Kubrick",
        "Año estreno": "1980"
      }
    },
    {
      "texto": "La película Titanic, lanzada en 1997 y dirigida por James Cameron, narra la trágica historia de amor entre Jack y Rose, dos pasajeros del famoso transatlántico que se hundió en su viaje inaugural.",
      "respuesta": {
        "Título": "Titanic",
        "Director": "James Cameron",
        "Año estreno": "1997"
      }
    },
    {
      "texto": "'Coco' es una animación de Disney Pixar que se sumerge en la cultura mexicana y sigue a un joven llamado Miguel, que sueña con convertirse en músico a pesar de la prohibición de su familia. La película fue estrenada en 2017.",
      "respuesta": {
        "Título": "Coco",
        "Director": "",
        "Año estreno": "2017"
      }
    },
    {
      "texto": "En 1994, El Rey León se convirtió en un clásico instantáneo de Disney, narrando la historia de Simba, un joven león que debe encontrar su lugar como rey de la sabana africana.",
      "respuesta": {
        "Título": "El Rey León",
        "Director": "",
        "Año estreno": "1994"
      }
    },
    {
      "texto": "*Pulp Fiction*, dirigida por Quentin Tarantino, es un entrelazado de historias sobre crimen y violencia en Los Ángeles. La película, estrenada en 1994, es conocida por su diálogo ingenioso y su estructura no lineal.",
      "respuesta": {
        "Título": "Pulp Fiction",
        "Director": "Quentin Tarantino",
        "Año estreno": "1994"
      }
    },
    {
      "texto": "La película Parásitos, ganadora del Óscar, sumerge al espectador en la vida de dos familias coreanas de extremos sociales opuestos que se entrelazan de manera inesperada. Su director, Bong Joon-ho, captura magistralmente la lucha de clases.",
      "respuesta": {
        "Título": "Parásitos",
        "Director": "Bong Joon-ho",
        "Año estreno": ""
      }
    },
    {
      "texto": "*La La Land*, un musical romántico lanzado en 2016, sigue la historia de una aspirante a actriz y un músico de jazz que luchan por hacer realidad sus sueños en Los Ángeles. La película fue dirigida por Damien Chazelle.",
      "respuesta": {
        "Título": "La La Land",
        "Director": "Damien Chazelle",
        "Año estreno": "2016"
      }
    },
    {
      "texto": "Un hito del cine mudo, Metrópolis, es una obra maestra de la ciencia ficción dirigida por Fritz Lang. Estrenada en 1927, la película presenta una futurística ciudad dividida entre trabajadores y la élite gobernante.",
      "respuesta": {
        "Título": "Metrópolis",
        "Director": "Fritz Lang",
        "Año estreno": "1927"
      }
    },
    {
      "texto": "*Interstellar*, una odisea espacial que explora los límites del amor y el sacrificio humano a través de un viaje interestelar en busca de un nuevo hogar para la humanidad, fue dirigida por Christopher Nolan y se estrenó en 2014.",
      "respuesta": {
        "Título": "Interstellar",
        "Director": "Christopher Nolan",
        "Año estreno": "2014"
      }
    },
    {
      "texto": "Mujercitas, de 2019, es una adaptación del clásico de Louisa May Alcott, diriga por Greta Gerwig, que sigue a las hermanas March en su paso de la infancia a la adultez",
      "respuesta": {
        "Título": "Mujercitas",
        "Director": "Greta Gerwig",
        "Año estreno": "2019"
      }
    },
    {
        "texto": "En 'Gravity' (2013), Sandra Bullock y George Clooney protagonizan como astronautas que luchan por sobrevivir después de que su transbordador espacial se destruye, dejándolos a la deriva en el espacio. Esta tensa y visualmente impresionante película explora la vastedad y el peligro del espacio exterior",
        "respuesta": {
            "Título": "Gravity",
            "Director": "",
            "Año estreno": "2013"
        }
    },
    {
        "texto": "El lobo de Wall Street narra la historia real de Jordan Belfort, interpretado por Leonardo DiCaprio, un corredor de bolsa de Nueva York que se enriquece a través de la corrupción y el fraude. La película, que también cuenta con la actuación de Jonah Hill y Margot Robbie, sumerge al espectador en el extravagante estilo de vida de Belfort y su eventual caída. Sin mencionar al director, la película captura el frenesí financiero de los años 90",
        "respuesta": {
            "Título": "El lobo de Wall Street",
            "Director": "",
            "Año estreno": ""
        }
    },
    {
        "texto": "Dirigida por Ridley Scott, *Blade Runner* es un clásico de ciencia ficción ambientado en el año 2019, donde Harrison Ford interpreta a Rick Deckard, un 'blade runner' encargado de cazar replicantes rebeldes. A pesar de su ambientación futurista en 2019, la película fue estrenada en 1982, convirtiéndose en una obra de culto por su profunda reflexión sobre la humanidad, la tecnología y el alma.",
        "respuesta": {
            "Título": "Blade Runner",
            "Director": "Ridley Scott",
            "Año estreno": "1982"
        }
    },
     {
    "texto": "\"Los Increíbles\", de Pixar, lanzada en 2004, es una animada aventura sobre una familia de superhéroes que intenta vivir una vida suburbana normal hasta que se ven obligados a volver a la acción.",
    "respuesta": {
      "Título": "Los Increíbles",
      "Director": "",
      "Año estreno": "2004"
    }
  },
  {
    "texto": "\"Forrest Gump\", protagonizada por Tom Hanks, es una emotiva historia de la vida de un hombre sencillo con un corazón puro y su extraordinaria jornada a través de eventos históricos de Estados Unidos en la segunda mitad del siglo XX. Estrenada en 1994.",
    "respuesta": {
      "Título": "Forrest Gump",
      "Director": "",
      "Año estreno": "1994"
    }
  },
  {
    "texto": "En \"El Club de la Pelea\", dirigida por David Fincher y lanzada en 1999, un empleado insomne y un vendedor de jabón carismático forman un club de lucha subterráneo que evoluciona en algo mucho más grande.",
    "respuesta": {
      "Título": "El Club de la Pelea",
      "Director": "David Fincher",
      "Año estreno": "1999"
    }
  },
  {
    "texto": "La comedia de 2014 \"El Gran Hotel Budapest\", dirigida por Wes Anderson, narra la historia de un conserje legendario en un famoso hotel europeo entre las guerras y su amistad con un joven empleado.",
    "respuesta": {
      "Título": "El Gran Hotel Budapest",
      "Director": "Wes Anderson",
      "Año estreno": "2014"
    }
  },
  {
    "texto": "\"Mad Max: Furia en la Carretera\", dirigida por George Miller en 2015, revive la franquicia con una persecución post-apocalíptica intensa y visualmente deslumbrante a través del desierto.",
    "respuesta": {
      "Título": "Mad Max: Furia en la Carretera",
      "Director": "George Miller",
      "Año estreno": "2015"
    }
  },
    {
    "texto": "\"El Silencio de los Inocentes\", lanzada en 1991, es un inquietante thriller psicológico donde una joven agente del FBI busca la ayuda de Hannibal Lecter, un brillante psiquiatra y asesino en serie encarcelado, para capturar a otro asesino.",
    "respuesta": {
      "Título": "El Silencio de los Inocentes",
      "Director": "",
      "Año estreno": "1991"
    }
  },
  {
    "texto": "El largometraje Gladiador, protagonizado por Russell Crowe y dirigida por Ridley Scott en 2000, presenta la épica historia de un general romano traicionado que se convierte en gladiador para vengar la muerte de su familia.",
    "respuesta": {
      "Título": "Gladiador",
      "Director": "Ridley Scott",
      "Año estreno": "2000"
    }
  },
  {
    "texto": "\"Amélie\", una película francesa de 2001, sigue a una joven inocente y soñadora en París que decide cambiar la vida de las personas a su alrededor para mejor mientras lucha con su propia soledad.",
    "respuesta": {
      "Título": "Amélie",
      "Director": "",
      "Año estreno": "2001"
    }
  },
  {
    "texto": "\"Jurassic Park\", dirigida por Steven Spielberg en 1993, revolucionó el cine con sus efectos especiales al traer dinosaurios a la vida en esta aventura emocionante en un parque temático de isla.",
    "respuesta": {
      "Título": "Jurassic Park",
      "Director": "Steven Spielberg",
      "Año estreno": "1993"
    }
  },
  {
    "texto": "\"Inception\" (Origen), dirigida por Christopher Nolan en 2010, es un thriller de ciencia ficción que explora el robo de secretos a través del subconsciente durante el sueño, llevando a cabo una idea compleja: la inserción de otra idea.",
    "respuesta": {
      "Título": "Inception",
      "Director": "Christopher Nolan",
      "Año estreno": "2010"
    }
  },
  {
    "texto": "La película italiana 'La Vida es Bella', de 1997, cuenta la historia de un padre judío que usa su imaginación para proteger a su hijo de los horrores de un campo de concentración nazi.",
    "respuesta": {
      "Título": "La Vida es Bella",
      "Director": "",
      "Año estreno": "1997"
    }
  }
]

In [292]:
evaluar_pipeline_con_conjunto_de_test(TEXTOS_TEST, pipeline_completo, model=model, tokenizer=tokenizer)

Esta es la respuesta del modelo en el pipeline:

['{\n    "Título": "El Resplandor",\n    "Director": "Stanley Kubrick",\n    "Año estreno": "1980"\n}']
Evaluando 1/24..
Resultado obtenido: {'Título': 'El Resplandor', 'Director': 'Stanley Kubrick', 'Año estreno': '1980'}
Resultado esperado: {'Título': 'El Resplandor', 'Director': 'Stanley Kubrick', 'Año estreno': '1980'}
Título correcto!
Director correcto!
Año de estreno correcto!
Tasa de acierto acumulada:100.0%
Esta es la respuesta del modelo en el pipeline:

['{\n    "Título": "Titanic",\n    "Director": "James Cameron",\n    "Año estreno": "1997"\n}']
Evaluando 2/24..
Resultado obtenido: {'Título': 'Titanic', 'Director': 'James Cameron', 'Año estreno': '1997'}
Resultado esperado: {'Título': 'Titanic', 'Director': 'James Cameron', 'Año estreno': '1997'}
Título correcto!
Director correcto!
Año de estreno correcto!
Tasa de acierto acumulada:100.0%
Esta es la respuesta del modelo en el pipeline:

['{\n    "Título": "Coco",\n    "Direct

0.8333333333333334